# WORDLE

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from tqdm import tqdm as ProgressDisplay
import math

### Lecture des fichiers de mots de 5 lettres

In [ ]:
# mots_1 = pd.read_json("mots.json")
# mots = pd.read_table("mots.txt",header=None) #version science etonnante
mots = pd.read_json("mots_off.json") #officielle depuis loane
data = mots.copy()
# alphabet= np.array(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'))
# alphabet= np.array(list('abcdefghijklmnopqrstuvwxyz'))
# frequence = {}
# for l in alphabet:
#     frequence[l]=0
# data.shape
dimension = data.size
print(dimension)

### Convertir un array en string

In [ ]:
def convert_array_mot(arr):
    return np.array([''.join(chr(e) for e in ligne )for ligne in arr])

* test de la fonction 

In [ ]:
liste = np.array([[101, 97, 116, 110, 97],[101, 97, 116, 110, 97]])
print(liste)
retour_mots = convert_array_mot(liste)
print(retour_mots)

### Convertir une string en array

In [ ]:
def convert_mot_array(mots):
    return np.array([[ord(c)for c in w] for w in mots], dtype=np.uint8)

In [ ]:
mots_array = convert_mot_array(data[0])
mots_array.shape


## Algorithm principal
 qui renvoie les options valides pour un mot, une pattern et une liste de mots
 * check : mot à tester
 * pattern : la pattern de type '01120'
 * M : le dictionnaire de mots

In [335]:
def check_mot_pattern(check,pattern,M=mots_array.copy(),k=5):
    # M = mots_array.copy()
    cnt = M.shape[0]
    for n in range(k):
        c =ord(check[n])
        if (pattern[n]=='0'):
            for i in range(k):
                M = M[~(M[:,i] == c)]
            # P = (np.sum(M[:] == c,axis=1)).reshape((1,M.shape[0]))
            # M = np.insert(M,0,P,axis=1)
            # M = M[(M[:,0] == 0)]
            # M = np.delete(M,0,axis=1)
        elif (pattern[n]=='2'):
            M = M[(M[:,n] == c)]
        else: # (pattern[n]=='1'):
            M = M[~(M[:,n] == c)] # on elimine la bonne postion
            P = (M[:] == c)
            T = (np.sum(P[:], axis=1)).reshape((1,M.shape[0]))
            M = np.insert(M,0,T,axis=1)
            M = M[(M[:,0] > 0)]
            M = np.delete(M,0,axis=1)
    return M,M.shape[0]

In [ ]:
M=mots_array.copy()
# print(M)
c=ord('E')
print(c,M.shape)
# print(M[~(M[:] == c)].shape)
P = (np.sum(M[:] == c,axis=1)).reshape((1,M.shape[0]))
M = np.insert(M,0,P,axis=1)
M = M[(M[:,0] == 0)]
M = np.delete(M,0,axis=1)
print(M.shape,M)


In [ ]:
ret = mots_array.copy()
ret, nb = check_mot_pattern('RIMES','00100',ret)
if (nb>0):
    entropie = -math.log(nb/dimension)/math.log(2)
else:
    entropie=0
print(nb,dimension,entropie,convert_array_mot(ret))

In [ ]:
def lettre_in(lettre,mot):
    return mot.find(lettre) 

## Creation de la pattern globale pour l'ensemble de 243 permutations (3^5)

In [ ]:
def make_pattern(p=5):
    pattern=[]
    for k in range(pow(3,5)):
        if (k<1):
            pattern.append(np.base_repr(k,3,5))
        elif (k<3):
            pattern.append(np.base_repr(k,3,4))
        elif (k<9):
            pattern.append(np.base_repr(k,3,3))
        elif (k<27):
            pattern.append(np.base_repr(k,3,2))
        elif (k<81):
            pattern.append(np.base_repr(k,3,1))
        else:
            pattern.append(np.base_repr(k,3))
    return pattern

In [ ]:
pattern = make_pattern(5)

## Calcul de l'Entropie

In [ ]:
def entropie_mot(check, M=mots_array.copy()):
    dim = M.shape[0]
    e_sum=0
    for p in pattern:
        ret, nb = check_mot_pattern(check,p,M)
        if (nb>0):
            e = -math.log(nb/dim)/math.log(2)
        else:
            e=0
        e_sum = e_sum + e*nb/dim
    return e_sum


# Calcul initial

Cela permet de déterminer dans 'liste_entropie' l'entropie de tous les mots dans le dictionnaire

L'entropie c'est : E = - log(p)/log(2)
ou p est la probabilité


In [ ]:
# initialement 9m05s ...
liste_entropie = {}
for mot in ProgressDisplay(data[0]):
    liste_entropie[mot] = entropie_mot(mot)
liste_entropie = sorted(liste_entropie.items(), key=lambda x: x[1],reverse=True)

In [ ]:
liste_entropie[:10]

## Aide au jeux
temps initial = 6,6s

Nous proposons RIMES
et calculons au préalable sont Entropie = 5,71971

nous jouons pour obtenir un résultat ⬛⬛🟨⬛⬛ soit '00100'

au final l'Entropie de ce choix est de 6,775 puisqu'il nous reste plus que 55 possibilités


In [ ]:
E = entropie_mot('RIMES')
print("Entropie de 'RIMES' = ",E)

In [ ]:
def conseil(check,pattern,M=mots_array.copy()):
    ret, nb = check_mot_pattern(check,pattern,M)
    E = - math.log(nb/M.shape[0])/math.log(2)
    new_liste = convert_array_mot(ret)
    # print(nb, E)
    new_entropie = {}
    for mot in ProgressDisplay(new_liste):
        new_entropie[mot] = entropie_mot(mot,ret)
    # print(f'Test : {new_entropie}')
    return(list(new_entropie))

In [ ]:
# 
ret, nb = check_mot_pattern('MOULE','00001',mots_array.copy())
E = - math.log(nb/dimension)/math.log(2)
new_liste = convert_array_mot(ret)
print(nb, E)
new_entropie = {}
for mot in ProgressDisplay(new_liste):
    new_entropie[mot] = entropie_mot(mot,ret)
new_entropie = sorted(new_entropie.items(), key=lambda x: x[1],reverse=True)


In [ ]:
print(dimension,ret.shape)
print(f'Test : {new_entropie[:20]}')

In [ ]:
# ret=mots_array.copy()
dim = ret.shape[0]
ret, nb = check_mot_pattern('PARTE','02101',ret)
print(nb)
E = - math.log(nb/dim)/math.log(2)
new_liste = convert_array_mot(ret)
print(nb, dim, E,new_liste)

# Simulation de jeux

* data[0] : liste de mots du dictionnaire
* mots_array : tableau des mots

In [ ]:
def test_mot(check,ref,k=5):
    rep=np.array([0,0,0,0,0])
    for i in range(k):
        if (check[i]==ref[i]):
            rep[i]=2
        else:
            if (lettre_in(check[i],ref) == -1):
                rep[i] = 0
            else:
                rep[i] = 1
    return rep

In [ ]:
def affiche_score(arr,k=5):
    rep=''
    for i in range(k):
        if (arr[i]==2):
            rep = rep + '🟩'
        if (arr[i]==1):
            rep = rep + '🟨'
        if (arr[i]==0):
            rep = rep + '⬛'
    return rep

In [ ]:
def tire_sort():
    ret = np.random.choice(data[0])
    return ret

In [ ]:
secret = tire_sort()
print('le mot secret est : ',secret)

In [ ]:
first='ORVET'
rep = test_mot(first,secret)
print (affiche_score(rep))

In [ ]:
ret, nb = check_mot_pattern(first,'10001',mots_array.copy())
# convert_array_mot(ret)

In [352]:
ret = check_mot_pattern('ORVET','00000')
print(ret)

(array([[65, 85, 83, 83, 73],
       [81, 85, 65, 78, 68],
       [65, 73, 78, 83, 73],
       ...,
       [81, 85, 73, 80, 85],
       [72, 73, 78, 68, 73],
       [83, 76, 65, 78, 71]], dtype=uint8), 629)


In [ ]:
conseil('ORVET','10001',ret)

In [ ]:
def toStr(arr):
    ret = ''
    for c in arr:
        ret = ret + str(c)
    return ret

In [310]:
rep = test_mot('TOITS',secret)
print(toStr(rep))
affiche_score(rep)

00001


'⬛⬛⬛⬛🟨'

In [346]:
conseil('PORTE','11000')

100%|██████████| 34/34 [00:01<00:00, 31.02it/s]


['JAPON',
 'HIPPO',
 'GALOP',
 'OPIUM',
 'JUPON',
 'CAMPO',
 'CHIPO',
 'YAPOK',
 'SIPOS',
 'CLOUP',
 'IPPON',
 'GLOUP',
 'APIOL',
 'CHOPS',
 'DISPO',
 'XIPHO',
 'CHOPA',
 'SUPPO',
 'FLOPS',
 'CAPOS',
 'CHAPO',
 'SCOPA',
 'VAPOS',
 'QUIPO',
 'SCOOP',
 'CAPON',
 'KAPOK',
 'SALOP',
 'KAPOS',
 'LAPON',
 'APION',
 'DIAPO',
 'OKAPI',
 'SLOOP']

In [343]:
data[0].searchsorted('MOULE')

2004

In [344]:
#simuler un jeu
def simul():
    secret = tire_sort()
    propal = 'RAIES' #plus forte Entropie
    filtre = mots_array.copy() # init avec tous le dictionnaire
    iter=0
    while (propal != secret):
        reponse = test_mot(propal,secret) # plus forte entropie
        filtre,nb = check_mot_pattern(propal,toStr(reponse),filtre)
        propal = conseil(propal,toStr(reponse),filtre)[0]
        print(iter, propal, secret, affiche_score(reponse))
        iter = iter +1
    return iter

In [309]:
jeux=np.array((0))
print(jeux)
for i in range(5):
    jeux = np.append(jeux,simul())
print('Moyenne des succès :',jeux.mean(axis=0))

0


100%|██████████| 45/45 [00:01<00:00, 41.62it/s]


0 ETRES OCRES 🟨⬛⬛🟩🟩


100%|██████████| 28/28 [00:00<00:00, 42.51it/s]


1 MERES OCRES 🟨⬛🟩🟩🟩


...

3 LASER JASER ⬛🟩🟩🟩🟩


100%|██████████| 3/3 [00:00<00:00, 41.70it/s]


4 JASER JASER ⬛🟩🟩🟩🟩


100%|██████████| 5/5 [00:00<00:00, 41.85it/s]

0 REINE REINE 🟩⬛🟩🟨⬛
Moyenne des succès : 3.0


In [ ]:
🟩⬛🟨⬛⬛

In [307]:
jeux.mean()


3.4752475247524752